In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import pickle
import os.path

from collections import defaultdict

# I. Класс реализованного метода

In [146]:
class UserBasedRecommender():
    def __init__(self, path_to_orders, path_to_order_products_train, decomposition_dim):
        
        self.path_to_orders = path_to_orders
        self.path_to_order_products_train = path_to_order_products_train
        self.n_comp = decomposition_dim
        
        self.ok_status = True
        check, message = self.check_input_params()
        
        if not check:
            self.ok_status = False
            print(message)
    
    def check_input_params(self):
        if not os.path.isfile(self.path_to_orders):
            return False, 'File doesnt exist'
        if not os.path.isfile(self.path_to_order_products_train):
            return False, 'File doesnt exist'
        if not isinstance(self.n_comp, int) or not self.n_comp > 0:
            return False, 'decomposition_dim should be positive integer value'
        
        return True, 'Ok'
            
    def read_data(self):
        if not self.ok_status:
            return 'There were some problems'
        
        self.orders = pd.read_csv(self.path_to_orders)
        self.order_products_train = pd.read_csv(self.path_to_order_products_train)
    
    def prepare_data(self):
        '''
        создает таблицу user-item для дальнейших манипуляций
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.merged = pd.merge(self.orders, self.order_products_train, on='order_id', how='right')
        self.merged = self.merged[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num = {key: value for value, key in enumerate(self.merged['user_id'].unique(), start=0)}
        self.product_id_num = {key: value for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        self.product_num_id = {value: key for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        
        self.merged['user_num'] = self.merged['user_id'].transform(lambda x: self.user_id_num[x])
        self.merged['product_num'] = self.merged['product_id'].transform(lambda x: self.product_id_num[x])
        self.merged['buy'] = 1
        
        self.user_item_matrix = sps.coo_matrix((self.merged.buy, (self.merged.user_num, self.merged.product_num)))
        
    def decompose(self):
        '''
        декомпозиция user-item_matrix с размерностью n_comp
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.model = NMF(n_components=self.n_comp, init='random', random_state=0)
        self.user_matrix = self.model.fit_transform(self.user_item_matrix)
        self.product_matrix = self.model.components_
    
    def get_similarity(self, top_n_size):
        '''Каждому юзеру сопоставляет id n-схожих юзеров'''
        if not self.ok_status:
            return 'There were some problems'
        
        self.similar_users_df = pd.DataFrame(columns=[i for i in range(1, top_n_size + 1)])

        for user_num in tqdm(range(len(self.user_matrix))):
          user_to_others_similarity = cosine_similarity(X=self.user_matrix, Y=self.user_matrix[user_num].reshape(1, self.n_comp))
          top_n = pd.DataFrame(user_to_others_similarity).sort_values(by=0, ascending = False)[:top_n_size].index
          self.similar_users_df.loc[user_num] = top_n
    
    def save_similarity_df(self, path):
        if not self.ok_status:
            return 'There were some problems'
        
        self.similar_users_df.to_csv(path)
        
    def set_similarity_df(self, path_to_df):
        '''
        Если уже есть посчитанная матрица, то можем использовать и обойти долгий кусок обучения в fit(),
        указав параметр loaded_df=True
        '''
        if not os.path.isfile(path_to_df):
            self.ok_status = False
            return 'File doesnt exist'
        
        self.similar_users_df = pd.read_csv(path_to_df)
    
    def get_predicted_user_item_matrix(self):
        pass
    
    def fit(self, path_to_save=False, loaded_df=False, top_n_size=5):
        '''
        :param self:
        :param path_to_save: путь для сохранения матрицы схожести
        :param loaded_df: bool - True если матрица уже была загружена и пересчитывать заново ее не нужно;
                          default=False
        :param top_n_size: размер топ схожих пользователей 
        :return: информация в случае ошибки 
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.read_data()     # читает необходимые df
        self.prepare_data()  # создание таблицы user-item с необх. полями и разметками
        self.decompose()     # декомпозиция матрицы на 2 матрицы размерности n
        if not loaded_df:
            self.get_similarity(top_n_size)
            if path_to_save:
                self.save_similarity_df(path_to_save)
        
    def predict(self, user_id):
        '''
        :param self:
        :param user_id:  id юзера, для которого считаем предсказание покупок
        :return: сообщение в случае ошибки; в случае успеха - словарь {product_id: сумма весов}
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        if user_id in self.user_id_num:
            user_num = self.get_user_num(user_id)
        else:
            return 'There is no user with this id'
        
        # индексы n_comp схожих пользователей
        sim_users_nums = self.similar_users_df.loc[user_num].values
        d = defaultdict(int)
        for sim_user_num in sim_users_nums:
            sim_user_purchases = pd.Series.sparse.from_coo(self.user_item_matrix)[sim_user_num].index
            sim_user_weight = cosine_similarity(X=self.user_matrix[sim_user_num].reshape(1, self.n_comp), Y=self.user_matrix[user_num].reshape(1, self.n_comp))
            
            for product in sim_user_purchases:
                product_id = self.product_num_id[product]
                d[product_id] += sim_user_weight
        
        return sorted(d.items(), key=lambda x: x[1], reverse=True)
    
    def get_coo(self):
        return self.user_item_matrix
    
    def get_user_num(self, user_id):
        return self.user_id_num[user_id]

## Пример работы с обучением

In [ ]:
# файл, куда по необходимости загрузим результаты
path_to_res = 'data/results.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 10

# top_n
top_n_size = 5

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Обучение с параметром path_to_save=filename после рассчета матрицы схожести сохранит df по указанному пути
# обучение длится долго, так что крайне советую этот параметр указать
Recommender.fit(path_to_save=path_to_res, top_n_size=top_n_size)

# можем пресказать что-то для юзера с id=12345 (пример выводы в следующей ячейке, так как матрица у меня уже была посчитана)
Recommender.predict(125)

# результат - номер продукта и пока просто сумма весов (те никак не нормализованное число, посчитано по матрице с 0/1)

## Пример работы без обучения (то есть уже есть просчитанная матрица - загрузим ее и сделаем предсказание):

In [149]:
# файл, где лежат результаты
path_to_res = 'data/results.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 10

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender.set_similarity_df(path_to_res)

# Обучение включает с параметром loaded_df=True создаст только необходимые для пресказания матрицы, 
# не относящиеся к уже имеющимся результатам (по дефолту False и для крохотных размерностей считалось 3 часа...)
Recommender.fit(loaded_df=True)

# можем пресказать что-то для юзера с id=125
Recommender.predict(125)

# результат - номер продукта и пока просто сумма весов (те никак не нормализованное число, посчитано по матрице с 0/1)

[(29487, array([[4.99052653]])),
 (19348, array([[2.99291994]])),
 (39646, array([[2.]])),
 (11827, array([[1.]])),
 (27676, array([[1.]])),
 (31564, array([[1.]])),
 (47502, array([[1.]])),
 (42284, array([[1.]])),
 (12620, array([[1.]])),
 (5460, array([[1.]])),
 (13866, array([[1.]])),
 (26965, array([[1.]])),
 (7877, array([[1.]])),
 (43056, array([[1.]])),
 (10613, array([[1.]])),
 (9755, array([[1.]])),
 (34726, array([[1.]])),
 (37543, array([[1.]])),
 (13287, array([[1.]])),
 (29176, array([[1.]])),
 (20463, array([[1.]])),
 (49079, array([[0.99796744]])),
 (38279, array([[0.99796744]])),
 (12911, array([[0.99796744]])),
 (18681, array([[0.99796744]])),
 (39282, array([[0.99796744]])),
 (48927, array([[0.99761497]])),
 (9387, array([[0.99760658]])),
 (41591, array([[0.99760658]])),
 (7736, array([[0.99760658]])),
 (432, array([[0.99760658]])),
 (13921, array([[0.99760658]])),
 (45835, array([[0.99760658]])),
 (39411, array([[0.99760658]])),
 (42441, array([[0.99760658]])),
 (27

# II. Пропишем все пути и сконструируем датасеты мечты

In [99]:
import pickle

# необходимые файлы с исходными датасетами с keggle
path_data_train = 'D:/Projects/Anaconda/EDA/data/order_products__train.csv'
path_data_prior = 'D:/Projects/Anaconda/EDA/data/order_products__prior.csv'
path_data_orders = 'D:/Projects/Anaconda/EDA/data/orders.csv'
    
# [куда запишем]/[где уже лежат] файлы с индексами
train_path = 'data_split/train_indices.pickle'
valid_path = 'data_split/validation_indices.pickle'
test_path = 'data_split/test_indices.pickle'

# куда запишем файлы с новым разделением
df_train_path = 'data_split/train_data.csv'
df_valid_path = 'data_split/validation_data.csv'
df_test_path = 'data_split/test_data.csv'

### Если файлов с индексами нет

In [102]:
def _split_indices(grouped_ratings, retriever):
    return np.concatenate(grouped_ratings.apply(retriever).values)

def split(path_to_orders):
    orders = pd.read_csv(path_to_orders)
    grouper = orders.sort_values('order_number').groupby('user_id')
    train_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings[:int(user_ratings.shape[0] * 0.5)].index.values)
    
    validation_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.5):
                                               int(user_ratings.shape[0] * 0.75)].index.values)
    
    test_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.75):].index.values)
    
    return train_indices, validation_indices, test_indices


train_indices, validation_indices, test_indices = split(path_data_orders)

# save results
with open(train_path, 'wb') as out:
    pickle.dump(train_indices, out)

with open(valid_path, 'wb') as out:
    pickle.dump(validation_indices, out)

with open(test_path, 'wb') as out:
    pickle.dump(test_indices, out)

### Файлы с индексами готовы - теперь скомпонуем и запишем в файл все необходимое

In [103]:
def read_and_concat_same_data(df_train_path, df_prior_path, df_orders_path):
    
    df_train = pd.read_csv(df_train_path)
    df_prior = pd.read_csv(df_prior_path)
    df_orders = pd.read_csv(df_orders_path)
    
    order_products_full = pd.concat([df_train, df_prior])
    
    return order_products_full, df_orders


def get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full):
    
    with open(train_path, 'rb') as input:
        train_indices = pickle.load(input)

    with open(valid_path, 'rb') as input:
        validation_indices = pickle.load(input)

    with open(test_path, 'rb') as input:
        test_indices = pickle.load(input)
        
    train_order_ids = df_orders.loc[train_indices]['order_id'].values
    validation_order_ids = df_orders.loc[validation_indices]['order_id'].values
    test_order_ids = df_orders.loc[test_indices]['order_id'].values
    
    train_df = df_order_products_full.loc[train_order_ids]
    validation_df = df_order_products_full.loc[validation_order_ids]
    test_df = df_order_products_full.loc[test_order_ids]
    
    return train_df, validation_df, test_df


# считается что уже имеем 3 файла с индексами
df_order_products_full, df_orders = read_and_concat_same_data(path_data_train, path_data_prior, path_data_orders)
train_df, validation_df, test_df = get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full)

# write new df's to file:
train_df.to_csv(df_train_path, index=False)
validation_df.to_csv(df_valid_path, index=False)
test_df.to_csv(df_test_path, index=False)

### notes


In [ ]:
# Лучше хранить в словаре, так как могут быть разные n для разных юзеров (как переделать текущий дф):
# d = df.to_dict('index')
# d[user_num].values() - выдает словарь список - можно пройтись в цикле и переписать

# в классе заюзаем dict и просто впишем массив, подфиксить под это надо predict(), fit(), set_similarity_df()

# учесть то, что для разных юзеров разное n:
# добавть параметр if sim<min_sim: отбрасываем юзера из топа => в таком случае n будет меньше стандартного

In [ ]:
h = pd.DataFrame(columns=[i for i in range(1, 3)])
d = h.to_dict('index')
for num in d[3].values():
    print(num)